In [1]:
import gdown
import os

url = "https://drive.google.com/drive/u/0/folders/1Oiq2ubkNc5xoN4dQDI3UYdYB1vGgYaJP"

gdown.download_folder(url, remaining_ok=True, use_cookies=False)
print("Attempted to download folder contents.")


Retrieving folder contents


Processing file 1RIgnFDnFbzMLyNLxT1AQEg7wlmNa2lZO Apple_10-Q4-2024-As-Filed.pdf
Processing file 1PrMab5gI8PcY-33-e4ishPJoqZUr1YVw Tesla_NASDAQ_TSLA_2023.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1RIgnFDnFbzMLyNLxT1AQEg7wlmNa2lZO
To: /content/ABB_LLM/Apple_10-Q4-2024-As-Filed.pdf
100%|██████████| 964k/964k [00:00<00:00, 30.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PrMab5gI8PcY-33-e4ishPJoqZUr1YVw
To: /content/ABB_LLM/Tesla_NASDAQ_TSLA_2023.pdf
100%|██████████| 985k/985k [00:00<00:00, 10.4MB/s]

Attempted to download folder contents.



Download completed


In [2]:
!pip install pymupdf
import pandas as pd
import fitz
import os
import re
!pip install langchain-text-splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter
!pip install chromadb sentence-transformers rfc3987
import chromadb
from chromadb.utils import embedding_functions
import uuid
print('ChromaDB and dependencies successfully installed and imported.')
!pip install -q sentence-transformers
from sentence_transformers import CrossEncoder
import chromadb

ChromaDB and dependencies successfully installed and imported.


In [3]:
# initialize global list to avoid duplicate entries from previous runs
!pip install pymupdf
import pandas as pd
import fitz
import os
import re
import pandas as pd
import fitz
import os
import re

structured_data = []

pdf_dir = '/content/ABB_LLM/'
file_paths = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
SECTION_HEADER_PATTERN = re.compile(r'(?i)^\s*(Item\s+\d+[A-Z]?)\b', re.MULTILINE)

def process_pdf_with_tables(file_path):
    filename = os.path.basename(file_path)
    doc = fitz.open(file_path)
    current_section = "Unknown"

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)

        # 1. Extract standard text
        raw_text = page.get_text("text")

        # 2. Extract tables and convert to Markdown strings
        table_texts = []
        tabs = page.find_tables()
        for tab in tabs:
            df_tab = tab.to_pandas()
            if not df_tab.empty:
                # Convert table to markdown format to preserve structure
                table_texts.append(df_tab.to_markdown(index=False))

        # 3. Combine text and tables
        combined_content = raw_text
        if table_texts:
            combined_content += "\n\n### Tables Extracted:\n" + "\n\n".join(table_texts)

        # 4. Cleaning
        cleaned_text = re.sub(r'\n{2,}', '\n\n', combined_content)
        cleaned_text = re.sub(r' +', ' ', cleaned_text).strip()

        # Detect section
        section_match = SECTION_HEADER_PATTERN.search(cleaned_text)
        if section_match:
            current_section = section_match.group(1).upper()

        structured_data.append({
            'filename': filename,
            'page_number': page_num + 1,
            'section': current_section,
            'content': cleaned_text
        })
    doc.close()

# Execute processing
for path in file_paths:
   process_pdf_with_tables(path)

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Initialize the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

enriched_chunks = []

# 2. Iterate through structured_data to split content
for page in structured_data:
    segments = text_splitter.split_text(page['content'])
    for segment in segments:
        enriched_chunks.append({
            "text": segment,
            "metadata": {
                "filename": page['filename'],
                "page_number": page['page_number'],
                "section": page['section']
            }
        })

# 3. Verify the process
print(f"Total enriched chunks generated: {len(enriched_chunks)}")
if enriched_chunks:
    print(f"Sample chunk metadata: {enriched_chunks[0]['metadata']}")

Total enriched chunks generated: 1168
Sample chunk metadata: {'filename': 'Apple_10-Q4-2024-As-Filed.pdf', 'page_number': 1, 'section': 'Unknown'}


In [5]:
import chromadb
from chromadb.utils import embedding_functions
import uuid
print('ChromaDB and dependencies successfully installed and imported.')

ChromaDB and dependencies successfully installed and imported.


In [6]:
# 1. Initialize persistent client using the Drive path
client = chromadb.PersistentClient(path='./chroma_db_storage')

# 2. Define embedding function
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name='all-MiniLM-L6-v2'
)

# 3. Delete existing collection if it exists
try:
    client.delete_collection(name='filings_appletesla')
    print("Deleted existing collection 'filings_appletesla'.")
except Exception as e:
    print(f"Collection not found or could not be deleted: {e}")

# 4. Create new collection
collection = client.create_collection(
    name='filings_appletesla',
    embedding_function=embedding_func
)

# 5. Prepare data for ingestion
documents = [chunk['text'] for chunk in enriched_chunks]
metadatas = [chunk['metadata'] for chunk in enriched_chunks]
ids = [str(uuid.uuid4()) for _ in range(len(enriched_chunks))]

# 6. Ingest data
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print(f"Successfully re-initialized collection '{collection.name}'")
print(f"Total records stored: {collection.count()}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Deleted existing collection 'filings_appletesla'.
Successfully re-initialized collection 'filings_appletesla' on Drive.
Total records stored: 1168


In [7]:
!pip install -q transformers accelerate

import transformers
import torch

print(f'Transformers version: {transformers.__version__}')

# Check for GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

if device == 'cuda':
    print(f'GPU Name: {torch.cuda.get_device_name(0)}')
else:
    print('Warning: No GPU detected. LLM inference will be slow on CPU.')

Transformers version: 5.0.0
Using device: cpu


In [8]:
# Define the custom RAG prompt template with strict citation enforcement
rag_prompt_template = """
Use the following pieces of retrieved context to answer the question.

--- INSTRUCTIONS ---
1. Use ONLY the information provided in the Context sources.
2. If the answer is not contained within the context (e.g. asking for future years not in the text), clearly state: \"Not Answerable\".
3. You MUST provide the answer followed by the exact citation in this format: [Filename, Section, Page X].
4. Do not write full sentences, just provide the data and the citation separating with comma.
5. If the context does not contain the specific year or data point requested, do not guess. Say \"Not Answerable\".
6. If the context does not contain specific information within the context(e.g. asking how are you) Say \"Not Answerable\" and dont provide any citation details.
7. CRITICAL: If the answer is \"Not Answerable\", do NOT provide any citation, headers, or source information.

--- CONTEXT ---
{context}

--- QUESTION ---
{query}

--- ANSWER ---
"""

print("RAG prompt template updated for strict validation.")

RAG prompt template updated for strict validation.


In [9]:
#avoid crashing of session and for memory efficient loading
!pip install -q bitsandbytes accelerate
import torch
import gc

def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

clear_memory()
print("Memory cleared and optimization libraries installed.")

Memory cleared and optimization libraries installed.


In [10]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoConfig, BitsAndBytesConfig

model_id = "microsoft/phi-2"
print("Loading Phi-2 optimized for CPU usage...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # 1. Config with padding fixed
    config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
    config.pad_token_id = tokenizer.pad_token_id

    # 2. Load model in 4-bit for memory efficiency
    # On CPU, bitsandbytes handles the dequantization overhead
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float32
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        config=config,
        quantization_config=bnb_config,
        device_map={ "": "cpu" },
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    rag_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
    )

    print("Phi-2 loaded successfully for CPU inference.")
except Exception as e:
    print(f"Error loading model: {e}")

def generate_rag_answer(query, ranked_chunks, top_n=1):
    # We use top_n=1 to minimize the number of tokens the CPU has to process
    context_segments = []
    for i, chunk in enumerate(ranked_chunks[:top_n]):
        source_info = f"[Source: {chunk['metadata']['filename']}, Section: {chunk['metadata']['section']}, Page: {chunk['metadata']['page_number']}]"
        context_segments.append(f"{source_info}\n{chunk['text']}")

    aggregated_context = "\n\n".join(context_segments)
    final_prompt = rag_prompt_template.format(context=aggregated_context, query=query)

    # Limiting max_new_tokens for faster CPU response
    sequences = rag_pipeline(
        final_prompt,
        max_new_tokens=50,
        do_sample=False,
        return_full_text=False,
        pad_token_id=tokenizer.pad_token_id
    )

    return sequences[0]['generated_text']

Loading Phi-2 optimized for CPU usage...


Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

Phi-2 loaded successfully for CPU inference.


In [11]:
import gc
import torch
import os

def optimize_session():
    """Clears memory and optimizes the environment for full execution."""
    # 1. Clear Python garbage collector
    gc.collect()

    # 2. Clear Torch cache (works for both CPU and GPU)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # 3. Inform the system to prioritize memory for the current process
    print("Memory optimization complete. Ready for sequential execution.")

# Run optimization
optimize_session()

Memory optimization complete. Ready for sequential execution.


### Tips for a Stable Session:
- **Sequential Execution**: Avoid running multiple intensive cells (like model loading and PDF processing) simultaneously.
- **Restart Runtime**: If you have been experimenting for a while, go to `Runtime > Restart session` before doing a 'Run All' to ensure a clean memory state.
- **Variable Reuse**: Avoid creating many large intermediate lists; for example, clear `structured_data` if you have already converted it into `enriched_chunks`.

In [12]:
import time
import torch

# 1. pre-loaded models
if 'rerank_model' not in globals():
    from sentence_transformers import CrossEncoder
    rerank_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device='cpu')

# 2. Re-defining the generation logic in one place
def generate_fast_answer(query, top_chunk):
    source_info = f"[Source: {top_chunk['metadata']['filename']}, Section: {top_chunk['metadata']['section']}, Page: {top_chunk['metadata']['page_number']}]"
    context = f"{source_info}\n{top_chunk['text']}"

    # Use the pre-defined strict RAG prompt
    final_prompt = rag_prompt_template.format(context=context, query=query)

    # Optimization for CPU: Increased max_new_tokens to 60 to ensure citation fits
    with torch.no_grad():
        sequences = rag_pipeline(
            final_prompt,
            max_new_tokens=60,
            do_sample=False,
            return_full_text=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    raw_answer = sequences[0]['generated_text']
    # Post-processing: remove model artifacts but keep the citation
    # We split by '---' and 'EXAMPLES' which are common markers where Phi-2 starts hallucinating
    clean_answer = raw_answer.split('---')[0].split('EXAMPLES')[0].strip()
    return clean_answer

# 3. Execution Block
start_time = time.time()
query = "What was Apples total revenue for the fiscal year ended September 28, 2024?"

# Retrieval
collection = client.get_collection(name='filings_appletesla', embedding_function=embedding_func)
results = collection.query(query_texts=[query], n_results=3)
candidates = results['documents'][0]
metadatas = results['metadatas'][0]

# Re-ranking
pairs = [[query, doc] for doc in candidates]
scores = rerank_model.predict(pairs)

ranked_results = []
for i in range(len(candidates)):
    ranked_results.append({"text": candidates[i], "metadata": metadatas[i], "score": scores[i]})
ranked_results = sorted(ranked_results, key=lambda x: x['score'], reverse=True)

# Generation
answer = generate_fast_answer(query, ranked_results[0])

duration = time.time() - start_time
print(f"\n--- RESULTS (Total Time: {duration:.2f}s) ---")
print(answer)

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'pad_token_id', 'do_sample', 'max_new_tokens', 'eos_token_id'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=30) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main


--- RESULTS (Total Time: 1533.35s) ---
$391,035


In [ ]:
import json
import os
from google.colab import _message

# 1. Configuration
GITHUB_USER = 'REDACTED'
GITHUB_REPO = "LLM_RAG"
GITHUB_TOKEN = 'REDACTED'
GITHUB_EMAIL = 'REDACTED'
notebook_filename = "RAG_Pipeline_Notebook.ipynb"

# 2. Get and Redact the Notebook
notebook_json = _message.blocking_request("get_ipynb", request="", timeout_sec=30)
nb_data = notebook_json["ipynb"]

for cell in nb_data.get("cells", []):
    if cell.get("cell_type") == "code":
        lines = cell.get("source", [])
        new_source = []
        for line in lines:
# Secret Redacted
                if "=" in line:
                    key = line.split("=")[0].strip()
                    new_source.append(f"{key} = 'REDACTED'\n")
                else:
                    new_source.append("# Secret Redacted\n")
            else:
                new_source.append(line)
        cell["source"] = new_source

with open(notebook_filename, "w", encoding="utf-8") as f:
    json.dump(nb_data, f, indent=1)

# 3. Git Operations for ALL files
# Secret Redacted
# Secret Redacted

# Wipe and re-init to ensure a clean push of everything
!rm -rf .git
!git init
!git add .
!git commit -m "Full repository push including redacted notebook and data files"

remote_url = 'REDACTED'
!git remote add origin {remote_url}
!git branch -M main
!git push -u origin main --force

print(f"\nSuccessfully pushed all files and {notebook_filename} to GitHub!")